Time Series Data Manipulation using Python

In [2]:
import pandas as pd

# Load the dataset
time_series_data = pd.read_csv(r"C:\Data Analysis\INR-USD.csv")

# Display the first few rows of the dataset
print(time_series_data.head())

         Date       Open       High        Low      Close  Adj Close  Volume
0   12/1/2003  45.709000  45.728001  45.449001  45.480000  45.480000     0.0
1   12/8/2003  45.474998  45.507999  45.352001  45.451000  45.451000     0.0
2  12/15/2003  45.450001  45.500000  45.332001  45.455002  45.455002     0.0
3  12/22/2003  45.417000  45.549000  45.296001  45.507999  45.507999     0.0
4  12/29/2003  45.439999  45.645000  45.421001  45.560001  45.560001     0.0


The dataset consists of columns like ‘Date’, ‘Open’, ‘High’, ‘Low’, ‘Close’, ‘Adj Close’, and ‘Volume’. These columns are typical in financial time series datasets, where:
Date: Represents the time at which the data was recorded.
Open: The price at the beginning of the trading period.
High: The highest price during the trading period.
Low: The lowest price during the trading period.
Close: The price at the end of the trading period.
Adj Close: The closing price after adjustments for all applicable splits and dividend distributions.
Volume: Trade Volume.
The next step is to ensure that the ‘Date’ column is correctly recognized as a date. It is crucial for time series analysis, as it allows the data to be manipulated and queried based on time. Let’s parse the ‘Date’ column and set it as the index of the DataFrame:


In [3]:
# Parsing the 'Date' column and setting it as the index
time_series_data['Date'] = pd.to_datetime(time_series_data['Date'])
time_series_data.set_index('Date', inplace=True)

# Display the DataFrame with the parsed dates
print(time_series_data.head())

                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2003-12-01  45.709000  45.728001  45.449001  45.480000  45.480000     0.0
2003-12-08  45.474998  45.507999  45.352001  45.451000  45.451000     0.0
2003-12-15  45.450001  45.500000  45.332001  45.455002  45.455002     0.0
2003-12-22  45.417000  45.549000  45.296001  45.507999  45.507999     0.0
2003-12-29  45.439999  45.645000  45.421001  45.560001  45.560001     0.0


Now that the ‘Date’ column is correctly parsed and set as the index, the next step is to inspect the data for any missing or anomalous values (using descriptive statistics):

In [4]:
# Checking for missing values
missing_values = time_series_data.isnull().sum()
print(missing_values)

Open         3
High         3
Low          3
Close        3
Adj Close    3
Volume       3
dtype: int64


In [5]:
basic_statistics = time_series_data.describe()
print(basic_statistics)

              Open         High          Low        Close    Adj Close  Volume
count  1013.000000  1013.000000  1013.000000  1013.000000  1013.000000  1013.0
mean     58.035208    58.506681    57.654706    58.056509    58.056509     0.0
std      12.614635    12.716632    12.565279    12.657407    12.657407     0.0
min      38.995998    39.334999    38.979000    39.044998    39.044998     0.0
25%      45.508999    45.775002    45.231998    45.498001    45.498001     0.0
50%      59.702999    60.342999    59.209999    59.840000    59.840000     0.0
75%      68.508499    69.099998    68.250000    68.538002    68.538002     0.0
max      82.917999    83.386002    82.563004    82.932999    82.932999     0.0


There are three missing values in each of the columns except for the ‘Date’ column. The descriptive statistics do not immediately suggest the presence of extreme anomalies or outliers.

In [6]:
# Handling missing values by imputing with the mean
time_series_data_filled = time_series_data.fillna(time_series_data.mean())

# Verify that there are no more missing values
missing_values_after = time_series_data_filled.isnull().sum()

print(missing_values_after)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


The missing values have been successfully filled with the mean of their respective columns, and there are no more missing values in the dataset.

In [7]:
# Resampling the data from weekly to monthly frequency
monthly_resampled_data = time_series_data_filled.resample('M').mean()

print(monthly_resampled_data.head())

                 Open       High        Low     Close  Adj Close  Volume
Date                                                                    
2003-12-31  45.498200  45.586000  45.370001  45.49080   45.49080     0.0
2004-01-31  45.365999  45.433750  45.172999  45.23675   45.23675     0.0
2004-02-29  45.167251  45.285999  45.106251  45.27450   45.27450     0.0
2004-03-31  45.026801  45.073800  44.558600  44.75880   44.75880     0.0
2004-04-30  43.767750  44.049250  43.630000  43.90800   43.90800     0.0


In [8]:
# Extracting date-time features
monthly_resampled_data['Year'] = monthly_resampled_data.index.year
monthly_resampled_data['Month'] = monthly_resampled_data.index.month
monthly_resampled_data['Quarter'] = monthly_resampled_data.index.quarter
monthly_resampled_data['WeekOfYear'] = monthly_resampled_data.index.isocalendar().week

# Creating lag features for 'Close' price
monthly_resampled_data['Close_Lag1'] = monthly_resampled_data['Close'].shift(1)
monthly_resampled_data['Close_Lag2'] = monthly_resampled_data['Close'].shift(2)

# Rolling window statistics for 'Close' price (using a window of 3 months)
monthly_resampled_data['Close_Rolling_Mean'] = monthly_resampled_data['Close'].rolling(window=3).mean()
monthly_resampled_data['Close_Rolling_Std'] = monthly_resampled_data['Close'].rolling(window=3).std()

# Displaying the DataFrame with the new features
print(monthly_resampled_data[['Year', 'Month', 'Quarter', 'WeekOfYear', 
                              'Close_Lag1', 'Close_Lag2', 
                              'Close_Rolling_Mean', 'Close_Rolling_Std']].head())

            Year  Month  Quarter  WeekOfYear  Close_Lag1  Close_Lag2  \
Date                                                                   
2003-12-31  2003     12        4           1         NaN         NaN   
2004-01-31  2004      1        1           5    45.49080         NaN   
2004-02-29  2004      2        1           9    45.23675    45.49080   
2004-03-31  2004      3        1          14    45.27450    45.23675   
2004-04-30  2004      4        2          18    44.75880    45.27450   

            Close_Rolling_Mean  Close_Rolling_Std  
Date                                               
2003-12-31                 NaN                NaN  
2004-01-31                 NaN                NaN  
2004-02-29           45.334017           0.137084  
2004-03-31           45.090017           0.287462  
2004-04-30           44.647100           0.690064  


Summary
Manipulating time series data involves several steps including loading the data, parsing time information, handling missing values, resampling for different time periods, and potentially transforming or aggregating the data.